<a href="https://colab.research.google.com/github/Adityaupadhyay013/Fruit-Freshness-classification-model/blob/main/Fruit_Freshness_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
path = kagglehub.dataset_download("abdulrafeyyashir/fresh-vs-rotten-fruit-images")
print("Path to dataset is: " , path)

100%|██████████| 38.5M/38.5M [00:00<00:00, 116MB/s]

Extracting files...


Path to dataset is:  /root/.cache/kagglehub/datasets/abdulrafeyyashir/fresh-vs-rotten-fruit-images/versions/4


In [2]:
import os
print(os.listdir(path))
file_path = f"{path}/Fruit Freshness Dataset"

['README.dataset.txt', 'Fruit Freshness Dataset']


In [3]:
filepath =  '/root/.cache/kagglehub/datasets/abdulrafeyyashir/fresh-vs-rotten-fruit-images/versions/4/Fruit Freshness Dataset/Fruit Freshness Dataset'
final_path = f"{file_path}/Fruit Freshness Dataset"
print(os.listdir(final_path))

['Strawberry', 'Banana', 'Apple']


In [4]:
import numpy as np
import pandas as pd
Straw_frame = pd.DataFrame()
fruits = {}
for items in os.listdir(final_path):
  f_name_file = f"{final_path}/{items}"
  for tags in os.listdir(f_name_file):
    if tags == 'Fresh':
      l_name_file = f"{f_name_file}/{tags}"
      for vals in os.listdir(l_name_file):
        Full_path = f"{l_name_file}/{vals}"
        fruits[Full_path] = 1
    else:
      l_name_file = f"{f_name_file}/{tags}"
      for vals in os.listdir(l_name_file):
        Full_path = f"{l_name_file}/{vals}"
        fruits[Full_path] = 0

In [5]:
Fruits = pd.DataFrame()
Fruits['img'] = fruits.keys()
Fruits['label'] = fruits.values()

In [6]:
Fruits

,img,label
0,/root/.cache/kagglehub/datasets/abdulrafeyyash...,1
1,/root/.cache/kagglehub/datasets/abdulrafeyyash...,1
2,/root/.cache/kagglehub/datasets/abdulrafeyyash...,1
3,/root/.cache/kagglehub/datasets/abdulrafeyyash...,1
4,/root/.cache/kagglehub/datasets/abdulrafeyyash...,1
...,...,...
561,/root/.cache/kagglehub/datasets/abdulrafeyyash...,0
562,/root/.cache/kagglehub/datasets/abdulrafeyyash...,0
563,/root/.cache/kagglehub/datasets/abdulrafeyyash...,0
564,/root/.cache/kagglehub/datasets/abdulrafeyyash...,0


### Read and Resize image + HSV + GLCM + LBP features extraction.

In [7]:
import cv2
from skimage.feature import graycomatrix , graycoprops , local_binary_pattern
def load_image(img_path , size = (128 , 128)):
  img = cv2.imread(img_path)
  img = cv2.resize(img , size)
  return img
def extract_HSV(img):
  hsv = cv2.cvtColor(img , cv2.COLOR_BGR2HSV)
  h , s , v = cv2.split(hsv)
  features = [
      np.mean(h) , np.std(h) ,
      np.mean(s) , np.std(s) ,
      np.mean(v) , np.std(v)
  ]
  return features
def extract_GLCM(img):
  gray = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
  glcm = graycomatrix(
      gray , distances = [1] , angles = [0]  , levels = 256 , symmetric = True , normed = True
  )
  contrast = graycoprops(glcm , 'contrast')[0 , 0]
  energy = graycoprops(glcm , 'energy')[0 , 0]
  homogeneity = graycoprops(glcm , 'homogeneity')[0 , 0]
  correlation = graycoprops(glcm , 'correlation')[0 , 0]
  return [contrast , energy , homogeneity , correlation]
def lbp_feature(img):
  gray = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY)
  lbp = local_binary_pattern(gray , P = 8 , R = 1 ,method = 'uniform')
  hist , _ = np.histogram(lbp , bins = 10 , range = (0 , 10))
  hist = hist.astype("float")
  hist /= (hist.sum()+1e-6)
  return hist.tolist()

In [8]:
def feature_extraction(img_path):
  img = load_image(img_path)
  color = extract_HSV(img)
  glcm = extract_GLCM(img)
  lbp = lbp_feature(img)
  return color+glcm+lbp

In [9]:
cols = [f"HSV_{i}" for i in range(6)] + [f"GLCM_{i}" for i in range(4)] + [f"LBP_{i}" for i in range(10)]
df = pd.DataFrame([] , columns = cols)

In [10]:
df

,HSV_0,HSV_1,HSV_2,HSV_3,HSV_4,HSV_5,GLCM_0,GLCM_1,GLCM_2,GLCM_3,LBP_0,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9


In [11]:
for img_path in Fruits['img']:
  features = feature_extraction(img_path)
  df.loc[len(df)] = features

In [12]:
df

,HSV_0,HSV_1,HSV_2,HSV_3,HSV_4,HSV_5,GLCM_0,GLCM_1,GLCM_2,GLCM_3,LBP_0,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9
0,13.770203,39.184377,70.080383,104.453716,232.392029,38.006173,466.136442,0.556624,0.644299,0.957636,0.020630,0.029724,0.018066,0.044800,0.108337,0.107239,0.027710,0.041931,0.544495,0.057068
1,24.162415,47.637720,76.058899,85.356478,233.209961,35.497471,844.259966,0.390077,0.464749,0.888180,0.050964,0.056519,0.027893,0.062866,0.072754,0.120544,0.036255,0.065613,0.402039,0.104553
2,25.615417,52.721846,72.172546,99.515252,212.963135,35.308661,879.723794,0.017508,0.073903,0.876862,0.093628,0.154419,0.043640,0.077698,0.104004,0.089600,0.056519,0.121033,0.125061,0.134399
3,23.967529,54.913863,64.032349,97.852089,226.132202,43.732424,567.245940,0.222231,0.529628,0.952628,0.038635,0.041138,0.019836,0.092712,0.094421,0.139832,0.044983,0.086426,0.365051,0.076965
4,46.369202,72.247184,74.354675,89.355023,219.934998,46.625438,844.146161,0.464406,0.515819,0.926032,0.040466,0.043335,0.030884,0.061035,0.076477,0.112122,0.035828,0.053955,0.463989,0.081909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,11.125854,31.601709,53.202942,70.958071,219.954834,51.798566,483.237943,0.418376,0.506354,0.947946,0.039185,0.044861,0.027527,0.073364,0.089600,0.110840,0.041565,0.069885,0.412415,0.090759
562,27.591370,49.495078,93.586548,46.110240,100.658875,58.095176,270.375431,0.035242,0.335895,0.940080,0.056152,0.066101,0.041443,0.101440,0.149963,0.195374,0.079895,0.078308,0.119141,0.112183
563,8.655762,32.482277,33.903809,62.949349,243.527588,27.780197,162.768701,0.696260,0.739711,0.970454,0.021301,0.023682,0.015869,0.033508,0.046814,0.077759,0.021301,0.032593,0.682251,0.044922
564,74.811707,82.833105,228.711914,34.112136,147.719299,69.637544,321.807087,0.017351,0.156047,0.946503,0.031433,0.043945,0.059692,0.161438,0.303711,0.153809,0.064453,0.054321,0.056030,0.071167


In [13]:
df['label'] = list(Fruits['label'])

In [ ]:
df

,HSV_0,HSV_1,HSV_2,HSV_3,HSV_4,HSV_5,GLCM_0,GLCM_1,GLCM_2,GLCM_3,...,LBP_1,LBP_2,LBP_3,LBP_4,LBP_5,LBP_6,LBP_7,LBP_8,LBP_9,label
0,90.093628,65.073508,68.746643,95.908327,212.623108,39.446754,265.402498,0.388397,0.656160,0.975737,...,0.028870,0.022034,0.059753,0.098511,0.161621,0.037170,0.047546,0.462280,0.057007,1
1,33.687622,64.979751,82.267639,95.645042,232.989868,32.984723,187.370571,0.476336,0.568323,0.982035,...,0.031372,0.025818,0.053711,0.107361,0.113708,0.040283,0.048157,0.476807,0.069031,1
2,11.029358,23.922529,105.541809,95.534383,227.876221,35.270295,421.754306,0.359930,0.421522,0.957001,...,0.056335,0.039856,0.066589,0.065735,0.110168,0.045532,0.069458,0.379822,0.109009,1
3,34.971252,66.002982,62.149292,91.398622,219.530518,60.509994,314.944513,0.561953,0.645040,0.978215,...,0.032043,0.023499,0.054016,0.092651,0.102417,0.029297,0.049377,0.542053,0.052368,1
4,97.521240,82.215358,115.037720,106.008781,194.529480,56.724828,258.086922,0.227948,0.443120,0.984168,...,0.042664,0.036560,0.091248,0.174316,0.163086,0.056091,0.061951,0.262817,0.081848,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,26.937866,33.387953,53.479309,36.354504,160.822632,40.919656,143.076956,0.032186,0.174959,0.954095,...,0.106934,0.057190,0.084961,0.104309,0.094238,0.067566,0.104248,0.117187,0.182312,0
562,11.341614,30.664885,22.439209,47.743441,227.711914,59.083581,254.648314,0.618180,0.735290,0.971050,...,0.018555,0.010681,0.084717,0.050537,0.111145,0.017578,0.089966,0.558655,0.045837,0
563,73.840332,44.010028,76.035095,61.252670,114.330200,47.597838,307.679072,0.016197,0.163408,0.918025,...,0.070312,0.066711,0.135986,0.205444,0.146057,0.076904,0.074036,0.067261,0.105774,0
564,2.799377,12.470800,17.192017,53.934611,245.068787,34.263756,346.690330,0.866101,0.880975,0.917887,...,0.009583,0.007202,0.020325,0.019409,0.061096,0.009277,0.015564,0.832275,0.018250,0


In [ ]:
df.isnull().sum()

,0
HSV_0,0
HSV_1,0
HSV_2,0
HSV_3,0
HSV_4,0
HSV_5,0
GLCM_0,0
GLCM_1,0
GLCM_2,0
GLCM_3,0


In [ ]:
Fruits[Fruits['label'] == 1].count()

,0
img,377
label,377


In [14]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 7.7 MB/s eta 0:00:00


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier , BaggingClassifier , GradientBoostingClassifier , AdaBoostClassifier , StackingClassifier
import xgboost
import optuna
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

### Preprocessing

In [16]:
Features = df.drop("label" , axis =1)
Target = df['label']

In [ ]:
Target

,label
0,1
1,1
2,1
3,1
4,1
...,...
561,0
562,0
563,0
564,0


In [17]:
x_train , x_test , y_train , y_test = train_test_split(Features , Target , test_size = 0.25 , random_state = 42)

In [ ]:
y_train

,label
108,1
228,1
565,0
429,0
410,0
...,...
71,1
106,1
270,1
435,0


In [ ]:
cols = list(x_train.columns)
print(cols)

['HSV_0', 'HSV_1', 'HSV_2', 'HSV_3', 'HSV_4', 'HSV_5', 'GLCM_0', 'GLCM_1', 'GLCM_2', 'GLCM_3', 'LBP_0', 'LBP_1', 'LBP_2', 'LBP_3', 'LBP_4', 'LBP_5', 'LBP_6', 'LBP_7', 'LBP_8', 'LBP_9']


In [18]:
T1 = ColumnTransformer([
    ("Scaler" , StandardScaler() , cols)
] , remainder = 'passthrough')

In [ ]:
Pipe = Pipeline([
    ("T1" , T1) ,
    ("LR" , LogisticRegression())
])

In [ ]:
Pipe.fit(x_train , y_train)

Pipeline(steps=[('T1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scaler', StandardScaler(),
                                                  ['HSV_0', 'HSV_1', 'HSV_2',
                                                   'HSV_3', 'HSV_4', 'HSV_5',
                                                   'GLCM_0', 'GLCM_1', 'GLCM_2',
                                                   'GLCM_3', 'LBP_0', 'LBP_1',
                                                   'LBP_2', 'LBP_3', 'LBP_4',
                                                   'LBP_5', 'LBP_6', 'LBP_7',
                                                   'LBP_8', 'LBP_9'])])),
                ('LR', LogisticRegression())])

In [ ]:
y_pred = Pipe.predict(x_test)

In [ ]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.95      0.83      0.88        46
           1       0.92      0.98      0.95        96

    accuracy                           0.93       142
   macro avg       0.94      0.90      0.92       142
weighted avg       0.93      0.93      0.93       142



In [ ]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 8.1 MB/s eta 0:00:00


In [ ]:
LR = LogisticRegression()
DTC = DecisionTreeClassifier()
RFC = RandomForestClassifier()
ADC = AdaBoostClassifier()
KNC = KNeighborsClassifier()
GBC = GradientBoostingClassifier()
Svc = SVC()
GNB = GaussianNB()
XGB = xgboost.XGBClassifier()

In [ ]:
from sklearn.metrics import accuracy_score
def Objective(trial):
  classifier_name = trial.suggest_categorical('classifier' , ['LR' , 'DTC' , 'RFC' , 'ADC' , 'KNC' , 'GBC' , 'Svc' , 'GNB' , 'XGB'])
  if classifier_name == 'LR':
    model = LR
  elif classifier_name == 'DTC':
    model = DTC
  elif classifier_name == 'RFC':
    model = RFC
  elif classifier_name == 'ADC':
    model = ADC
  elif classifier_name == 'KNC':
    model = KNC
  elif classifier_name == 'GBC':
    model = GBC
  elif classifier_name == 'Svc':
    model = Svc
  elif classifier_name == 'GNB':
    model = GNB
  elif classifier_name == 'XGB':
    model = XGB
  model.fit(x_train , y_train)
  y_pred = model.predict(x_test)
  return accuracy_score(y_test , y_pred)

In [ ]:
study = optuna.create_study(direction = 'maximize' , sampler = optuna.samplers.TPESampler())
study.optimize(Objective , n_trials = 50)

[I 2026-01-18 09:11:17,325] A new study created in memory with name: no-name-ba3ba9b5-1df7-435c-be60-6453fd6ef4de
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[I 2026-01-18 09:11:17,435] Trial 0 finished with value: 0.9084507042253521 and parameters: {'classifier': 'LR'}. Best is trial 0 with value: 0.9084507042253521.
[I 2026-01-18 09:11:17,492] Trial 1 finished with value: 0.9154929577464789 and parameters: {'classifier': 'DTC'}. Best is trial 1 with value: 0.9154929577464789.
[I 2026-01-18 09:11:17,533] Trial 2 finished with 

In [ ]:
print(study.best_value)
print(study.best_params)

0.9577464788732394
{'classifier': 'GBC'}


### From Hyperparameter tuning using Optuna it is clear that GradientBoostingClassifier is best algorithm for this dataset.
Untuned GBC accuracy = 0.9577464

In [ ]:
Pipe = Pipeline([
    ("T1" , T1) ,
    ("GBC" , GBC)
])

In [ ]:
Pipe.fit(x_train , y_train)

Pipeline(steps=[('T1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scaler', StandardScaler(),
                                                  ['HSV_0', 'HSV_1', 'HSV_2',
                                                   'HSV_3', 'HSV_4', 'HSV_5',
                                                   'GLCM_0', 'GLCM_1', 'GLCM_2',
                                                   'GLCM_3', 'LBP_0', 'LBP_1',
                                                   'LBP_2', 'LBP_3', 'LBP_4',
                                                   'LBP_5', 'LBP_6', 'LBP_7',
                                                   'LBP_8', 'LBP_9'])])),
                ('GBC', GradientBoostingClassifier())])

In [ ]:
y_pred = Pipe.predict(x_test)

In [ ]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92        46
           1       0.97      0.96      0.96        96

    accuracy                           0.95       142
   macro avg       0.94      0.95      0.94       142
weighted avg       0.95      0.95      0.95       142



### Hyperparameter tuning in GradientBoostingClassifier

In [ ]:
def objective(trial):
  loss = trial.suggest_categorical('loss' , ['log_loss' , 'exponential'])
  learning_rate = trial.suggest_float('learning_rate' , 0.1 , 0.9 , step = 0.2)
  n_estimators = trial.suggest_int('n_estimators' , 100 ,500 , step = 100)
  subsample = trial.suggest_float('subsample' , 0.1 ,0.9 , step = 0.2)
  criterion = trial.suggest_categorical('criterion', ['friedman_mse' , 'squared_error'])
  min_samples_split = trial.suggest_int('min_samples_split' , 2 , 5 , step = 1)
  max_depth = trial.suggest_int('max_depth' , 2 ,5 , step = 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf' , 1 , 5 , step = 1)
  max_features = trial.suggest_categorical('max_features' , [None , 5 , 10 , 20])
  model = GradientBoostingClassifier(loss = loss , learning_rate = learning_rate , n_estimators = n_estimators , subsample = subsample ,
                        criterion = criterion , min_samples_split =  min_samples_split , max_depth = max_depth , min_samples_leaf = min_samples_leaf ,
                                   max_features = max_features)
  model.fit(x_train , y_train)
  y_pred = model.predict(x_test)
  return accuracy_score(y_test , y_pred)

In [ ]:
Study = optuna.create_study(direction = 'maximize' , sampler = optuna.samplers.TPESampler())
Study.optimize(objective , n_trials = 50)

[I 2026-01-18 10:04:03,400] A new study created in memory with name: no-name-8f8af29f-524d-4235-a99e-ee0235faa455
[I 2026-01-18 10:04:03,961] Trial 0 finished with value: 0.5774647887323944 and parameters: {'loss': 'log_loss', 'learning_rate': 0.5, 'n_estimators': 400, 'subsample': 0.1, 'criterion': 'friedman_mse', 'min_samples_split': 2, 'max_depth': 2, 'min_samples_leaf': 2, 'max_features': 20}. Best is trial 0 with value: 0.5774647887323944.
[I 2026-01-18 10:04:06,634] Trial 1 finished with value: 0.9647887323943662 and parameters: {'loss': 'log_loss', 'learning_rate': 0.30000000000000004, 'n_estimators': 500, 'subsample': 0.9, 'criterion': 'squared_error', 'min_samples_split': 3, 'max_depth': 4, 'min_samples_leaf': 1, 'max_features': 20}. Best is trial 1 with value: 0.9647887323943662.
[I 2026-01-18 10:04:07,698] Trial 2 finished with value: 0.9507042253521126 and parameters: {'loss': 'exponential', 'learning_rate': 0.5, 'n_estimators': 100, 'subsample': 0.5, 'criterion': 'squared_

In [ ]:
print(Study.best_value)
print(Study.best_params)

0.971830985915493
{'loss': 'log_loss', 'learning_rate': 0.7000000000000001, 'n_estimators': 300, 'subsample': 0.9, 'criterion': 'squared_error', 'min_samples_split': 5, 'max_depth': 4, 'min_samples_leaf': 4, 'max_features': 5}


###### After Hyperparameter tuning in GradientBoostingClassifier i got best params: {'loss': 'log_loss', 'learning_rate': 0.7000000000000001, 'n_estimators': 300, 'subsample': 0.9, 'criterion': 'squared_error', 'min_samples_split': 5, 'max_depth': 4, 'min_samples_leaf': 4, 'max_features': 5} ,
best score i am getting is: 0.971830985915493.


In [24]:
model1 = GradientBoostingClassifier(loss =  'log_loss', learning_rate =  0.7000000000000001, n_estimators =  300, subsample =  0.9, criterion = 'squared_error', min_samples_split =  5, max_depth =  4, min_samples_leaf =  4, max_features =  5)
Pipe = Pipeline([
    ("T1" , T1) ,
    ("TUGBC" , model1)
])

In [26]:
Pipe.fit(x_train , y_train)

Pipeline(steps=[('T1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scaler', StandardScaler(),
                                                  ['HSV_0', 'HSV_1', 'HSV_2',
                                                   'HSV_3', 'HSV_4', 'HSV_5',
                                                   'GLCM_0', 'GLCM_1', 'GLCM_2',
                                                   'GLCM_3', 'LBP_0', 'LBP_1',
                                                   'LBP_2', 'LBP_3', 'LBP_4',
                                                   'LBP_5', 'LBP_6', 'LBP_7',
                                                   'LBP_8', 'LBP_9'])])),
                ('TUGBC',
                 GradientBoostingClassifier(criterion='squared_error',
                                            learning_rate=0.7000000000000001,
                                            max_depth=4, max_features=5,
                                            min_samples_leaf=4,
                                            min_samples_split=5,
                                            n_estimators=300, subsample=0.9))])

In [27]:
y_pred = Pipe.predict(x_test)

In [28]:
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.93      0.82      0.88        51
           1       0.91      0.97      0.94        91

    accuracy                           0.92       142
   macro avg       0.92      0.90      0.91       142
weighted avg       0.92      0.92      0.91       142



#### Save the current model

In [42]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [43]:
import joblib
joblib.dump(Pipe , "/content/drive/MyDrive/Fruit Freshness Classification model.pkl")

['/content/drive/MyDrive/Fruit Freshness Classification model.pkl']

In [44]:
import joblib
Pipe = joblib.load("/content/drive/MyDrive/Fruit Freshness Classification model.pkl")
type(Pipe)

sklearn.pipeline.Pipeline

In [ ]:
y_test.head(1)

,label
539,0


In [ ]:
sample = x_test.head(1)
actual = y_test.head(1)
print("sample is: " , sample , "Actual is: " , actual)
pred = Pipe.predict(sample)
print("Predicted is: " , pred[0] , "Actual is: " , actual)
print(pred[0] == actual)

sample is:           HSV_0      HSV_1      HSV_2      HSV_3       HSV_4      HSV_5  \
539  25.083435  29.668677  129.43573  50.477488  171.606445  54.159062   

          GLCM_0    GLCM_1    GLCM_2    GLCM_3     LBP_0     LBP_1     LBP_2  \
539  1135.495325  0.009872  0.065237  0.792548  0.074158  0.096191  0.068176   

        LBP_3    LBP_4     LBP_5    LBP_6     LBP_7    LBP_8     LBP_9  
539  0.118469  0.16156  0.095642  0.05719  0.083984  0.09375  0.150879   Actual is:  539    0
Name: label, dtype: int64
Predicted is:  0 Actual is:  539    0
Name: label, dtype: int64
539    True
Name: label, dtype: bool


### Now i think i should go with Hyperparameter tuned xgboostclassifier for best possible accuracy.

In [22]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def obj(trial):
  booster = trial.suggest_categorical('booster' , ['gbtree' , 'gblinear' , 'dart'])
  max_depth = trial.suggest_int('max_depth' , 3 , 10)
  min_child_weight = trial.suggest_int('min_child_weight' , 1 , 10)
  gamma = trial.suggest_int('gamma' , 0 , 10 , step = 2)
  max_leaves = trial.suggest_int('max_leaves' , 0 , 256)
  grow_policy = trial.suggest_categorical('grow_policy' , ['depthwise' , 'lossguide'])
  eta = trial.suggest_float('eta' , 0.01 , 0.3)
  n_estimators = trial.suggest_int('n_estimators' , 100 , 1000)
  subsample = trial.suggest_float('subsample' , 0.1 , 0.99)
  model = XGBClassifier(booster = booster , max_depth = max_depth , min_child_weight = min_child_weight , gamma = gamma , max_leaves = max_leaves ,
                  grow_policy = grow_policy , eta = eta , n_estimators = n_estimators ,
                  subsample = subsample)
  model.fit(x_train , y_train)
  y_pred = model.predict(x_test)
  return accuracy_score(y_test , y_pred)

In [ ]:
Study = optuna.create_study(direction = 'maximize' , sampler = optuna.samplers.TPESampler())
Study.optimize(obj , n_trials = 50)

[I 2026-01-18 17:33:51,371] A new study created in memory with name: no-name-eeec6af7-a04d-4f4d-aac1-31bfcb9ccfa6
[I 2026-01-18 17:33:52,159] Trial 0 finished with value: 0.9366197183098591 and parameters: {'booster': 'gbtree', 'max_depth': 5, 'min_child_weight': 10, 'gamma': 6, 'max_leaves': 176, 'grow_policy': 'depthwise', 'eta': 0.11875298399844691, 'n_estimators': 272, 'subsample': 0.3695878502022528}. Best is trial 0 with value: 0.9366197183098591.
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [17:33:52] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "gamma", "grow_policy", "max_depth", "max_leaves", "min_child_weight", "subsample" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[I 2026-01-18 17:33:52,407] Trial 1 finished with value: 0.9014084507042254 and parameters: {'booster': 'gblinear', 'max_depth': 3, 'min_child_weight': 6, 'gamma': 10, 'max_leaves': 255, 'grow_policy': 'lossguide', 'eta': 0.11498876809193813, 'n_estimato

In [ ]:
print(Study.best_value)
print(Study.best_params)

0.9647887323943662
{'booster': 'dart', 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0, 'max_leaves': 232, 'grow_policy': 'lossguide', 'eta': 0.20617613151894496, 'n_estimators': 494, 'subsample': 0.9428305777466978}


In [31]:
model2 = XGBClassifier(booster =  'dart', max_depth = 3, min_child_weight = 5, gamma = 0, max_leaves = 232, grow_policy = 'lossguide', eta = 0.20617613151894496, n_estimators = 494, subsample = 0.9428305777466978)
Pipe = Pipeline([
    ("T1" , T1) ,
    ("TUXGB" , model2)
])

In [ ]:
Pipe.fit(x_train , y_train)

Pipeline(steps=[('T1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scaler', StandardScaler(),
                                                  ['HSV_0', 'HSV_1', 'HSV_2',
                                                   'HSV_3', 'HSV_4', 'HSV_5',
                                                   'GLCM_0', 'GLCM_1', 'GLCM_2',
                                                   'GLCM_3', 'LBP_0', 'LBP_1',
                                                   'LBP_2', 'LBP_3', 'LBP_4',
                                                   'LBP_5', 'LBP_6', 'LBP_7',
                                                   'LBP_8', 'LBP_9'])])),
                ('TUXGB',
                 XGBClassifier(base_score=None, booster='dart', callbacks=None,
                               colsam...
                               feature_types=None, feature_weights=None,
                               gamma=0, grow_policy='lossguide',
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=232, min_child_weight=5,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=494,
                               n_jobs=None, ...))])

In [ ]:
y_pred = Pipe.predict(x_test)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.98      0.91      0.94        46
           1       0.96      0.99      0.97        96

    accuracy                           0.96       142
   macro avg       0.97      0.95      0.96       142
weighted avg       0.97      0.96      0.96       142



              precision    recall  f1-score   support

           0       0.98      0.91      0.94        46
           1       0.96      0.99      0.97        96

    accuracy                           0.96       142
   macro avg       0.97      0.95      0.96       142
weighted avg       0.97      0.96      0.96       142



In [ ]:
import sklearn, xgboost, numpy
print(sklearn.__version__)
print(xgboost.__version__)
print(numpy.__version__)


1.6.1
3.1.2
2.0.2


In [ ]:
Target

,label
0,1
1,1
2,1
3,1
4,1
...,...
561,0
562,0
563,0
564,0


In [45]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(Pipe , Features , Target , n_jobs = -1)

In [46]:
print("Scores is = " , scores)
print("Actual accuracy is: " , scores.mean())

Scores is =  [0.88596491 0.96460177 0.96460177 0.84070796 0.82300885]
Actual accuracy is:  0.8957770532526004


Scores is =  [0.8245614  0.91150442 0.92035398 0.90265487 0.7699115 ]
Actual accuracy is:  0.8657972364539667

Scores is =  [0.87719298 0.95575221 0.97345133 0.80530973 0.82300885]
Actual accuracy is:  0.8869430212699891

My current model:

Scores is =  [0.88596491 0.96460177 0.96460177 0.84070796 0.82300885]
Actual accuracy is:  0.8957770532526004


#### Voting Classifier

In [ ]:
estms = [('TGB' , model1) , ('TXGB' , model2)]
VC = VotingClassifier(estimators = estms)

In [ ]:
Pipe = Pipeline([
    ("T1" , T1) ,
    ("VC" , VC)
])

In [ ]:
Pipe.fit(x_train , y_train)

Pipeline(steps=[('T1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scaler', StandardScaler(),
                                                  ['HSV_0', 'HSV_1', 'HSV_2',
                                                   'HSV_3', 'HSV_4', 'HSV_5',
                                                   'GLCM_0', 'GLCM_1', 'GLCM_2',
                                                   'GLCM_3', 'LBP_0', 'LBP_1',
                                                   'LBP_2', 'LBP_3', 'LBP_4',
                                                   'LBP_5', 'LBP_6', 'LBP_7',
                                                   'LBP_8', 'LBP_9'])])),
                ('VC',
                 VotingClassifier(estimators=[('TGB',
                                               GradientBoostingClassifier(criter...
                                                             feature_types=None,
                                                             feature_weights=None,
                                                             gamma=0,
                                                             grow_policy='lossguide',
                                                             importance_type=None,
                                                             interaction_constraints=None,
                                                             learning_rate=None,
                                                             max_bin=None,
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=3,
                                                             max_leaves=232,
                                                             min_child_weight=5,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=494,
                                                             n_jobs=None, ...))]))])

In [ ]:
y_pred = Pipe.predict(x_test)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92        46
           1       0.97      0.96      0.96        96

    accuracy                           0.95       142
   macro avg       0.94      0.95      0.94       142
weighted avg       0.95      0.95      0.95       142



#### BAGGING V/S VOTING V/S STACKING

In [19]:
def Objec(trial):
  classifier_name = trial.suggest_categorical('classifier' , ['VTC' , 'BGC' , 'STC'])
  if classifier_name == 'VTC':
   voting = trial.suggest_categorical('voting' , ['hard' , 'soft'])
   weights = trial.suggest_categorical('weights' , [(1 , 5) , (5 , 1) , (1 , 6) , (6 , 1) , (1 , 7) , (7 , 1) , (1 , 8) , (8 , 1)])
   model = VotingClassifier(estimators = [('TGB' , model1) , ('TXGB' , model2)] , voting = voting , weights = weights)
  elif classifier_name == 'BGC':
    n_estimators = trial.suggest_int('n_estimators' , 5 , 200 , step = 5)
    max_samples = trial.suggest_int('max_samples' , 100 , 200 , step = 20)
    model = BaggingClassifier(estimator = model1 , n_estimators = n_estimators , max_samples = max_samples , bootstrap = True ,oob_score = False)
  else:
    final_estimators = trial.suggest_categorical('final_estimators' , ['RandomForestClassifier' , 'LogisticRegression' , 'DecisionTreeClassifier'])
    if final_estimators == 'RandomForestClassifier':
      final_model = RandomForestClassifier()
    elif final_estimators == 'LogisticRegression':
      final_model = LogisticRegression()
    else:
      final_model = DecisionTreeClassifier()
    stack_method = trial.suggest_categorical('stack_method' , ['auto', 'predict_proba' , 'predict'])
    model = StackingClassifier(estimators = [('TGB' , model1) , ('TXGB' , model2)] , final_estimator = final_model ,
                               stack_method = stack_method)
  model.fit(x_train , y_train)
  y_pred = model.predict(x_test)
  return accuracy_score(y_test , y_pred)

In [32]:
Study = optuna.create_study(direction = 'maximize' , sampler = optuna.samplers.TPESampler())
Study.optimize(Objec , n_trials = 50 , catch = (ZeroDivisionError))

[I 2026-01-19 18:02:27,213] A new study created in memory with name: no-name-c93a585d-9bec-42a7-a960-ec5d04f25581
[I 2026-01-19 18:02:44,989] Trial 0 finished with value: 0.9154929577464789 and parameters: {'classifier': 'BGC', 'n_estimators': 85, 'max_samples': 120}. Best is trial 0 with value: 0.9154929577464789.
[I 2026-01-19 18:03:15,287] Trial 1 finished with value: 0.9366197183098591 and parameters: {'classifier': 'STC', 'final_estimators': 'LogisticRegression', 'stack_method': 'predict_proba'}. Best is trial 1 with value: 0.9366197183098591.
[I 2026-01-19 18:03:44,388] Trial 2 finished with value: 0.9507042253521126 and parameters: {'classifier': 'STC', 'final_estimators': 'LogisticRegression', 'stack_method': 'predict_proba'}. Best is trial 2 with value: 0.9507042253521126.
/tmp/ipython-input-1350291303.py:5: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (1, 5) which is of type tuple.


In [33]:
print(Study.best_value)
print(Study.best_params)

0.9507042253521126
{'classifier': 'STC', 'final_estimators': 'LogisticRegression', 'stack_method': 'predict_proba'}


#### 0.9647887323943662
#### {'classifier': 'VTC', 'voting': 'soft', 'weights': (1, 5)}

In [ ]:
model = VotingClassifier(estimators = [('TGB' , model1) , ('TXGB' , model2)] , voting = 'soft' , weights = (1 , 5))

In [36]:
Pipe = Pipeline([
    ("T1" , T1) ,
    ("STC" , StackingClassifier(estimators = [('TGB' , model1) , ('TXGB' , model2)] , final_estimator = LogisticRegression() , stack_method = 'predict_proba'))
])

In [37]:
Pipe.fit(x_train , y_train)

Pipeline(steps=[('T1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scaler', StandardScaler(),
                                                  ['HSV_0', 'HSV_1', 'HSV_2',
                                                   'HSV_3', 'HSV_4', 'HSV_5',
                                                   'GLCM_0', 'GLCM_1', 'GLCM_2',
                                                   'GLCM_3', 'LBP_0', 'LBP_1',
                                                   'LBP_2', 'LBP_3', 'LBP_4',
                                                   'LBP_5', 'LBP_6', 'LBP_7',
                                                   'LBP_8', 'LBP_9'])])),
                ('STC',
                 StackingClassifier(estimators=[('TGB',
                                                 GradientBoostingClassifier(cri...
                                                               importance_type=None,
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=3,
                                                               max_leaves=232,
                                                               min_child_weight=5,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               multi_strategy=None,
                                                               n_estimators=494,
                                                               n_jobs=None, ...))],
                                    final_estimator=LogisticRegression(),
                                    stack_method='predict_proba'))])

In [38]:
y_pred = Pipe.predict(x_test)
print(classification_report(y_test , y_pred))

              precision    recall  f1-score   support

           0       0.96      0.88      0.92        51
           1       0.94      0.98      0.96        91

    accuracy                           0.94       142
   macro avg       0.95      0.93      0.94       142
weighted avg       0.94      0.94      0.94       142

